<a href="https://colab.research.google.com/github/machtg/movie-recommendation-bangkit/blob/master/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation